In [10]:
import datetime
import pandas as pd

import pickle

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import preprocessing

In [11]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [12]:
MODEL_FOLDER = "model"

PROCESSED_DATA_FOLDER = "data/4_all_data_preprocessed"
ISW_DATA_FILE = "all_isw.csv"
WEATHER_EVENTS_DATA_FILE = "all_hourly_weather_events_v2.csv"

OUTPUT_DATA_FILE = "all_features"

tfidf_transformer_model = "tfidf_transformer"
count_vectorizer_model = "count_vectorizer"

tfidf_transformer_version = "v4"
count_vectorizer_version = "v4"

In [13]:
def isNaN(num):
    return num != num

### Load preprocessed data

In [14]:
# Load CSVs
df_isw = pd.read_csv(f"{PROCESSED_DATA_FOLDER}/{ISW_DATA_FILE}", sep=";")
df_weather_events = pd.read_csv(f"{PROCESSED_DATA_FOLDER}/{WEATHER_EVENTS_DATA_FILE}", sep=";")

print(df_isw.shape)

# Load TF-IDF
tfidf = pickle.load(open(f"{MODEL_FOLDER}/{tfidf_transformer_model}_{tfidf_transformer_version}.pkl", "rb"))
cv = pickle.load(open(f"{MODEL_FOLDER}/{count_vectorizer_model}_{count_vectorizer_version}.pkl", "rb"))

(402, 7)


### Prepare weather & events

In [15]:
# Technicaly it's feature engineering

# Replace timestamps with numerical values
df_weather_events.day_sunset = df_weather_events.day_sunset.apply(lambda x: 
    (datetime.datetime.strptime(x, "%H:%M:%S") - datetime.datetime.strptime("00:00:00", "%H:%M:%S")).total_seconds()
)
df_weather_events.day_sunrise = df_weather_events.day_sunrise.apply(lambda x: 
    (datetime.datetime.strptime(x, "%H:%M:%S") - datetime.datetime.strptime("00:00:00", "%H:%M:%S")).total_seconds()
)
df_weather_events.hour_datetime = df_weather_events.hour_datetime.apply(lambda x:
    (datetime.datetime.strptime(x, "%H:%M:%S") - datetime.datetime.strptime("00:00:00", "%H:%M:%S")).total_seconds()//3600
)

# Encode categories
le = preprocessing.LabelEncoder()
df_weather_events.hour_conditions = le.fit_transform(df_weather_events.hour_conditions)
df_weather_events.hour_preciptype = le.fit_transform(df_weather_events.hour_preciptype)

# Create new column with boolean event indicator
df_weather_events["event_indicator"] = df_weather_events.event_hour_level_event_datetimeEpoch.apply(lambda x: not isNaN(x))

# Drop now irrelevant features
features_to_drop = [
    "day_datetimeEpoch",
    "hour_datetimeEpoch",
    "event_all_region",
    "event_clean_end",
    "event_intersection_alarm_id",
    "event_start_time",
    "event_end_time",
    "event_start_hour",
    "event_end_hour",
    "event_day_date",
    "event_start_hour_datetimeEpoch",
    "event_end_hour_datetimeEpoch",	
    "event_hour_level_event_time",
    "event_hour_level_event_datetimeEpoch",
    "event_region_title",
    "event_region_city",
    "event_start",
    "event_end",
    "city",
    "region",
    "center_city_ua",
    "center_city_en",
    "region_alt"
]
df_weather_events = df_weather_events.drop(features_to_drop, axis=1)

# Convert everything to float
df_weather_events.hour_preciptype = df_weather_events.hour_preciptype.astype(float)
df_weather_events.hour_conditions = df_weather_events.hour_conditions.astype(float)
df_weather_events.region_id = df_weather_events.region_id.astype(float)
df_weather_events.event_holiday_is_near = df_weather_events.event_holiday_is_near.apply(lambda x: 1.0 if x and not isNaN(x) else 0.0)

# Fill NAN
df_weather_events.fillna(0)

print(df_weather_events.dtypes)
df_weather_events.sample(10)

city_resolvedAddress            object
day_datetime                    object
day_tempmax                    float64
day_tempmin                    float64
day_temp                       float64
day_dew                        float64
day_humidity                   float64
day_precip                     float64
day_precipcover                float64
day_solarradiation             float64
day_solarenergy                float64
day_uvindex                    float64
day_sunrise                    float64
day_sunset                     float64
day_moonphase                  float64
hour_datetime                  float64
hour_temp                      float64
hour_humidity                  float64
hour_dew                       float64
hour_precip                    float64
hour_precipprob                float64
hour_snow                      float64
hour_snowdepth                 float64
hour_preciptype                float64
hour_windgust                  float64
hour_windspeed           

,city_resolvedAddress,day_datetime,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_solarradiation,day_solarenergy,day_uvindex,day_sunrise,day_sunset,day_moonphase,hour_datetime,hour_temp,hour_humidity,hour_dew,hour_precip,hour_precipprob,hour_snow,hour_snowdepth,hour_preciptype,hour_windgust,hour_windspeed,hour_winddir,hour_pressure,hour_visibility,hour_cloudcover,hour_solarradiation,hour_solarenergy,hour_uvindex,hour_severerisk,hour_conditions,region_id,event_alarms_past_24,event_simultaneous_alarms,event_hours_from_last_alarm,event_holiday_is_near,event_indicator
295,"Луцьк, Луцький район, Україна",2022-03-08,0.6,-3.9,-0.8,-4.9,74.5,0.163,4.17,113.2,9.7,4.0,24497.0,65497.0,0.18,1.0,-0.7,86.96,-2.6,0.0,0.0,0.0,0.1,5.0,20.9,11.9,334.8,1018.0,21.8,91.7,0.0,NaN,0.0,10.0,3.0,3.0,NaN,NaN,NaN,0.0,False
119193,"Запоріжжя, Україна",2022-11-04,11.1,6.4,8.4,5.4,82.0,0.200,4.17,30.5,2.6,2.0,23286.0,58625.0,0.32,11.0,9.3,69.95,4.1,0.0,0.0,0.0,0.0,5.0,20.2,9.0,1.2,1017.0,24.1,60.4,88.0,0.3,1.0,10.0,4.0,8.0,NaN,NaN,NaN,0.0,False
82546,"Харків, Україна",2022-06-12,29.0,17.0,20.6,15.8,75.4,0.600,8.33,136.7,11.7,4.0,15951.0,74659.0,0.42,21.0,18.0,77.46,14.0,0.0,0.0,0.0,0.0,5.0,11.9,10.8,20.0,1013.0,10.0,0.0,9.0,0.0,0.0,3.0,0.0,20.0,NaN,NaN,NaN,0.0,False
37174,"Херсон, Україна",2022-03-11,-2.6,-7.6,-4.9,-9.9,68.6,0.500,20.83,117.7,10.2,3.0,22235.0,64164.0,0.28,21.0,-4.1,85.95,-6.1,0.0,0.0,0.0,0.3,5.0,27.7,19.1,12.9,1025.0,13.1,84.6,0.0,NaN,0.0,10.0,4.0,21.0,NaN,NaN,NaN,0.0,False
48410,"Чернівці, Україна",2022-07-20,30.5,14.7,23.2,11.5,51.7,0.000,0.00,322.7,28.0,9.0,20190.0,76090.0,0.70,20.0,28.6,32.82,10.6,0.0,0.0,0.0,0.0,5.0,41.3,17.3,332.0,1015.4,15.8,10.0,183.0,0.7,2.0,10.0,0.0,24.0,NaN,NaN,NaN,0.0,False
36884,"Херсон, Україна",2022-02-27,7.0,-0.1,3.2,-2.0,70.4,0.000,0.00,156.5,13.6,4.0,23588.0,63131.0,0.87,19.0,3.7,62.97,-2.7,0.0,0.0,0.0,0.0,5.0,22.0,15.5,28.2,1028.0,24.1,99.5,169.0,0.6,0.0,10.0,3.0,21.0,NaN,NaN,NaN,0.0,False
119101,"Запоріжжя, Україна",2022-10-31,11.4,4.1,8.5,2.7,68.2,0.100,4.17,107.1,9.3,5.0,22918.0,58997.0,0.19,22.0,4.5,66.50,-1.2,0.0,0.0,0.0,0.0,5.0,10.4,4.7,308.4,1020.0,24.1,0.0,0.0,NaN,0.0,10.0,0.0,8.0,7.0,4.0,14.469444,0.0,True
176676,"Черкаси, Україна",2022-10-26,15.0,11.9,13.1,11.1,88.2,0.100,4.17,46.7,3.9,3.0,26997.0,63645.0,0.01,15.0,15.0,74.97,10.6,0.0,0.0,0.0,0.0,5.0,37.8,22.7,310.5,1015.0,24.1,51.5,116.0,0.4,1.0,10.0,4.0,23.0,NaN,NaN,NaN,0.0,False
107943,"Ужгород, Ужгородський район, Україна",2022-07-10,19.7,13.8,16.3,8.5,60.3,1.278,8.33,281.2,24.4,9.0,20364.0,77559.0,0.36,17.0,17.7,57.41,9.2,0.0,0.0,0.0,0.0,5.0,34.9,10.9,357.0,1014.2,32.2,64.4,608.0,2.2,6.0,10.0,4.0,7.0,NaN,NaN,NaN,0.0,False
306,"Луцьк, Луцький район, Україна",2022-03-08,0.6,-3.9,-0.8,-4.9,74.5,0.163,4.17,113.2,9.7,4.0,24497.0,65497.0,0.18,12.0,0.6,56.33,-7.1,0.0,0.0,0.0,0.3,5.0,40.3,22.3,345.3,1019.0,22.2,100.0,221.0,0.8,2.0,10.0,3.0,3.0,2.0,1.0,153.944444,0.0,True


### Calculate TF-IDF matrix

In [16]:
# Calculate word count vector for each report (save as matrix)
word_count_vector = cv.transform(df_isw["lemm"].values.astype("U"))
# print("Word count vector:\n", word_count_vector)

# Calculate global tfidf vectors matrix
tfidf_vector = tfidf.transform(word_count_vector)

# transform tfidf matrix to sparse dataframe
df_tfidf_vector = pd.DataFrame.sparse.from_spmatrix(tfidf_vector, columns=cv.get_feature_names_out())
df_tfidf_vector = pd.concat([df_isw.date_tomorrow_datetime, df_tfidf_vector], axis=1)

print(df_tfidf_vector.shape)

(402, 7282)


### Merge final dataset

In [17]:
# Merge tfidf matrix with weather events dataset (takes 1m to execute)
df_final = df_weather_events.merge(df_tfidf_vector.add_prefix("isw_"),
                                    how="left",
                                    left_on="day_datetime",
                                    right_on="isw_date_tomorrow_datetime")
print(df_final.shape)
df_final.sample(5)

(195084, 7323)


city_resolvedAddress day_datetime  day_tempmax  day_tempmin   
92842   Хмельницька область, Україна   2022-08-13         24.7         16.9  \
124351                Рівне, Україна   2022-07-03         24.4         19.6   
174692              Черкаси, Україна   2022-08-06         28.7         19.4   
83231                Харків, Україна   2022-07-08         32.0         18.0   
135878              Житомир, Україна   2022-10-30         14.7         10.3   

        day_temp  day_dew  day_humidity  day_precip  day_precipcover   
92842       20.3     17.6          85.0       4.000             4.17  \
124351      21.6     18.9          85.1       0.009             4.17   
174692      23.9     19.2          76.5       0.000             0.00   
83231       25.4     12.5          52.0       0.000             0.00   
135878      12.3      8.9          79.7       0.000             0.00   

        day_solarradiation  day_solarenergy  day_uvindex  day_sunrise   
92842                242.9             21.1          8.0      21607.0  \
124351               127.0             11.1          5.0      18492.0   
174692               293.4             25.3          8.0      19783.0   
83231                319.8             27.6          9.0      16524.0   
135878                83.5              7.6          4.0      24716.0   

        day_sunset  day_moonphase  hour_datetime  hour_temp  hour_humidity   
92842      73977.0           0.51           16.0       23.2          72.11  \
124351     77398.0           0.13           22.0       21.6          83.06   
174692     73484.0           0.27           14.0       27.0          63.16   
83231      74667.0           0.30           11.0       30.0          28.92   
135878     60320.0           0.15            2.0       12.8          89.40   

        hour_dew  hour_precip  hour_precipprob  hour_snow  hour_snowdepth   
92842       17.9          0.0              0.0        0.0             0.0  \
124351      18.6          0.0              0.0        0.0             0.0   
174692      19.4          0.0              0.0        0.0             0.0   
83231       10.0          0.0              0.0        0.0             0.0   
135878      11.1          0.0              0.0        0.0             0.0   

        hour_preciptype  hour_windgust  hour_windspeed  hour_winddir   
92842               3.0           25.9            10.4          75.1  \
124351              5.0            4.0             2.2          24.2   
174692              5.0           22.0            12.6          60.2   
83231               5.0           22.7            10.8         150.0   
135878              5.0           30.6            16.6         286.3   

        hour_pressure  hour_visibility  hour_cloudcover  hour_solarradiation   
92842          1012.0              5.0            100.0                601.0  \
124351         1019.0             24.1              0.0                  3.0   
174692         1015.0             24.1             44.5                815.0   
83231          1015.0             10.0              0.6                784.0   
135878         1017.0             24.1            100.0                  0.0   

        hour_solarenergy  hour_uvindex  hour_severerisk  hour_conditions   
92842                2.2           6.0             10.0              3.0  \
124351               0.0           0.0             10.0              0.0   
174692               2.9           8.0             10.0              4.0   
83231                2.8           8.0             10.0              0.0   
135878               NaN           0.0             10.0              3.0   

        region_id  event_alarms_past_24  event_simultaneous_alarms   
92842        22.0                   NaN                        NaN  \
124351       17.0                   NaN                        NaN   
174692       23.0                   NaN                        NaN   
83231        20.0                   NaN                        NaN   
135878        6.

In [18]:
# Save final dataset
with open(f"{PROCESSED_DATA_FOLDER}/{OUTPUT_DATA_FILE}.pkl", 'wb') as handle:
    pickle.dump(df_final, handle)

In [19]:
"""y = df_final.event_indicator.astype(float)
df_final = df_final.drop(["isw_date_tomorrow_datetime", "day_datetime", "city_resolvedAddress", "event_indicator"], axis=1)

y.head(5)"""

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: event_indicator, dtype: float64